In [26]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

In [27]:
from transformers import pipeline

In [21]:
class AbstractClassifier(object):
        
        def __init__(self,model_name,tokenizer_name,token_type):
            self.classifier = pipeline("token-classification",model=model_name,tokenizer=tokenizer_name)
            self.token_type = token_type
        # it will return a index_buf
        # item in index_buf explain
        # index_buf[0]: word start index
        # index_buf[1]: word end index
        # index_buf[2]: word types
        def get_sentence_index(self,sentence):
            dic_result = self.classifier(sentence)

            index_buf = []
            count = -1
            for item in dic_result:
                if( 'b-' in item['entity'].lower()):
                    count += 1
                    index_buf.append([item['start'],item['end'],self.token_type])
                if('i-' in item['entity'].lower()):
                    index_buf[count][1] = item['end']
            return index_buf


In [ ]:

disease_classifier = AbstractClassifier(model_name="alvaroalon2/biobert_diseases_ner",
                                        tokenizer_name="alvaroalon2/biobert_diseases_ner",
                                        token_type="disease")
chemical_classifier = AbstractClassifier(model_name="sschet/biobert_chemical_ner",
                                         tokenizer_name="sschet/biobert_chemical_ner",
                                         token_type="chemical")
genetic_classifier = AbstractClassifier(model_name="alvaroalon2/biobert_genetic_ner",
                                        tokenizer_name="alvaroalon2/biobert_genetic_ner",
                                        token_type="genetic")

sentence = "Hepatitis B virus (Hepatitis B) is the pathogen that causes hepatitis B (hepatitis B for short)."
sentence += "Nitrofurans are broad-spectrum antibiotics that kill most gram-positive and Gram-negative bacteria, fungi, protozoa and other pathogens. They act on microbial enzyme system, inhibit acetyl-coA, interfere with microbial carbohydrate metabolism, and thus play A antibacterial role."
sentence += "Fibrinogen is one of the earliest coagulation factors found by human beings. It is an elongated ellipsoid. It is a dimer formed by three pairs of polypeptide chains (a pair of α chains, a pair of β chains, a pair of γ chains) connected by disulfide bonds, and its molecular weight is about 340,000."
index_bufs = []
index_bufs.append(disease_classifier.get_sentence_index(sentence))
index_bufs.append(chemical_classifier.get_sentence_index(sentence))
index_bufs.append(genetic_classifier.get_sentence_index(sentence))

In [25]:
for index_buf in index_bufs:
    for index_pair in index_buf:
        print("word:\""+ sentence[index_pair[0]:index_pair[1]] + "\" type:" + index_pair[2])

word:"Hepatitis B" type:disease
word:"Hepatitis B" type:disease
word:"hepatitis B" type:disease
word:"hepatitis B" type:disease
word:"Nitrofuran" type:chemical
word:"acetyl-coA" type:chemical
word:"carbohydrate" type:chemical
word:"disulfide" type:chemical
word:"Fibrinogen" type:genetic
